[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/openvinotoolkit/openvino_tensorflow/blob/master/examples/notebooks/Colab_OpenVINO_TensorFlow_classification_example.ipynb)

# **Image Classification with OpenVINO™ integration with TensorFlow**:

OpenVINO™ integration with TensorFlow is designed for TensorFlow developers who want to get started with OpenVINO™ in their inferencing applications. This product effectively delivers OpenVINO™ inline optimizations which enhance inferencing performance with minimal code modifications. OpenVINO™ integration with TensorFlow accelerates inference across many AI models on a variety of Intel® silicon such as: 
*   Intel® CPUs
*   Intel® integrated GPUs
*   Intel® Movidius™ Vision Processing Units - referred to as VPU
*   Intel® Vision Accelerator Design with 8 Intel Movidius™ MyriadX VPUs - referred to as VAD-M or HDDL

**Overview**

The following code demonstrates acceleration of InceptionV3 using OpenVINO™ integration with TensorFlow. We compare the performance of InceptionV3 with and without OpenVINO™ integration with TensorFlow.


InceptionV3 is a convolutional neural network for assisting in image analysis and object detection. 
We load a pre-trained version of this network trained on more than a million images 
from the ImageNet database.

## INSTALL OpenVINO™ integration with TensorFlow


In [ ]:
TF_VERSION = "2.7.0"
OVTF_VERSION = "1.1.0"

In [ ]:
# Upload the required wheel files, models and images in a google drive folder
# Uncomment and run the below command to copy them in your current workspace
#!cp /content/drive/MyDrive/TF-OV/working_dir_files/* . 

import os

!python3 -m pip install --upgrade pip
!python3 -m pip -q install pillow
!python3 -m pip -q install -U tensorflow_hub

# Install TensorFlow and OpenVINO-TensorFlow only if they aren't found
!if python3 -c "import tensorflow"; then echo "Found TensorFlow. Skipping."; else echo "TensorFlow Not Found. Installing."; python3 -m pip -q install tensorflow=={TF_VERSION}; fi
!if python3 -c "import openvino_tensorflow"; then echo "Found OpenVINO-TensorFlow. Skipping."; else echo "OpenVINO-TensorFlow Not Found. Installing."; python3 -m pip -q install openvino-tensorflow=={TF_VERSION}; fi

# Now lets infer

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
from IPython.display import HTML

import numpy as np
import tensorflow as tf
import openvino_tensorflow as ovtf
import tensorflow_hub as hub

from PIL import Image
import time

In [ ]:
def preprocess_image(file_name,
               input_height=299,
               input_width=299,
               input_mean=0,
               input_std=255):
    """Reads input image file, resizes it to given input height and width
       and returns the pre-processed image
    """  
    image = Image.open(file_name)
    resized_image = image.resize((input_height,input_width))
    resized_image = np.asarray(resized_image, np.float32)
    normalized_image = (resized_image - input_mean) / input_std
    result = np.expand_dims(normalized_image, 0)
    return result

In [ ]:
def load_labels(label_file):
    """Parses the label file, assuming that labels are separated with a newline
       in the file and returns the list of labels.
    """  
    label = []
    proto_as_ascii_lines = tf.io.gfile.GFile(label_file).readlines()
    for l in proto_as_ascii_lines:
        label.append(l.rstrip())
    return label

In [ ]:
def infer_openvino_tensorflow(model_file, file_name , input_height, input_width, input_mean, input_std, label_file):
    """Takes the tensorflow model and all other input parameters as arguments. 
       Runs inference with the classification model and prints the predictions.
    """
    print("CREATE MODEL - BEGIN")
    if model_file=="":
        model = hub.load("https://tfhub.dev/google/imagenet/inception_v3/classification/4")
    else:
        model = tf.saved_model.load(model_file)
    print("CREATE MODEL - END")

    print("PREDICTION - BEGIN") 

    img =  tf.convert_to_tensor(preprocess_image(
            file_name, input_height=input_height, input_width=input_width, input_mean=input_mean, input_std=input_std))

    # Warmup
    results = model(img)
    # Run
    
    for num_times in range(10):
        start = time.time()
        results = model(img)
        elapsed = time.time() - start
        print('Inference time in ms: %f' % (elapsed * 1000))
            
    print("PREDICTION - END")
    results = tf.nn.softmax(results).numpy()
    
    if label_file:
        top_5 = tf.argsort(results, axis=-1, direction="DESCENDING")[0][:5].numpy()
        labels = load_labels(label_file)
        for i,item in enumerate(top_5):
            print(labels[item], results[0][top_5][i])
    else:
        print("No label file provided. Cannot print classification results")

*   Set all the parameters needed for inference
*   Enable OpenVINO™ integration with TensorFlow, and set Backend in just a few simple lines of code to boost performace
*   Infer the input image 
*   Output the top 5 predicted classes, and the inference time with OpenVINO™ integration with TensorFlow enabled

In [ ]:
file_name = tf.keras.utils.get_file(
    'grace_hopper.jpg',
    "https://www.tensorflow.org/images/grace_hopper.jpg")
model_file = ""
label_file = tf.keras.utils.get_file(
    'ImageNetLabels.txt',
    'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
input_height = 299
input_width = 299
input_mean = 0
input_std = 255
backend_name = "CPU"
    

#Print list of available backends
print('Available Backends:')
backends_list = ovtf.list_backends()
for backend in backends_list:
    print(backend)
ovtf.set_backend(backend_name)

print("OpenVINO TensorFlow is enabled")
infer_openvino_tensorflow(model_file, file_name, input_height, input_width, input_mean, input_std, label_file)

*   Disable OpenVINO™ integration with TensorFlow to gauge the achieved performance boost
*   Infer the input image again
*   Output the top 5 predicted classes, and the inference time with OpenVINO™ integration with TensorFlow disabled


In [ ]:
ovtf.disable() ## Disabling OVTF
print("OpenVINO TensorFlow is disabled")
infer_openvino_tensorflow(model_file, file_name, input_height, input_width, input_mean, input_std, label_file )
ovtf.enable()